## Simulate non-cyanogenic evolution via drift alone

There is a finding that across a cline from less to more urban the frequency of a non-cyanogenic phenotype increases
The phenotype is genetically controlled by two loci, both of which have a segregating knock-out allele
If any individual is homozygous for either knockout they become non-cyanogenic

In this extremely simple simulation I create a 'population' which is represented by 2 lists of alleles (A/a and B/b)
To simulate evolution I randomly sample with replacement from the lists to create new list that represent the next generation.

By repeating this process with variable starting frequencies, population sizes and numbers of generations (functionally equivalent to steps in a strict stepping stone) we can look at the change in the frequency of cyanogenic and non-cyanogenic phenotypes

## Extension to previous simulations — Spatial structure

To build off of the previous single-population stepping stone model, I have added a few functions that allow us to add spatial structure to our simulations, effectively simulating the trajectory of multiple populations simultaneously. Once migration is added, this will be analogous to a metapopulation.

#### Major changes to script

1. Since there will be multiple populations, they are now stored as entries in a dictionary. Each key in the dictionarry corresponds to a population and the value is a list containing the frequencies of alleles phenotype, updated each generation. 
2. The alleles that correspond to each population are stored in a separate dictionary with keys matching those from the population dictionary. As such, allele frequencies for populations are calculated by sampling the allele lists in the 'alleles' dictionary with the key matching the population.
3. Every generation, each population has some probability (p) of generating a new population with N alleles sampled from the population that created it. This is not the most realistic scenario but can easily be modified later. 
4. Simulations are now stored separately in a dictionary called 'sim' where each key corresponds to a simulation. 
5. These changes allow us to model clines in two ways:
    - Clines across 'time': looking at phenotype frequencies **within** populations
    - Clines across 'space': looking at phenotype frequencies **across** created populations

## Extension to previous simulations — Logistic population growth

To build off of the simulations that included spatial structure, I have incorporated logistic population growth into the simulations. 

**Major changes to script**

1. Newly created populations form as a bottleneck from the population that created it. The proportion of alleles sampled is some fraction (bot) multiplied by the population size of the source population. 
2. Every generation, every population grows in size according to an approximate logistic growth curve. The logistic growth function incorporates a fractional constant that, when multiplied by the current population size, modifies the per capita growth rate of the population. The carrying capacity is thus altered by changing this fractional constant. 
3. The size of every population is stored as an integer in a list within the alleles dictionary under the key corresponding to the population. This integer is called as needed by the functions that require population size as an argument (e.g. 'bottle', 'pop_growth').

## Extension to previous simulations — Migration

To simulations that previously included spatial structure and logistic population growth, I have added migration

**Major changes to script**

1. Every generation, every population exchanges alleles with every other population at a rate that declines with the distance between them. Migration rate declines linearly with distance and is determined by the 'migration_rate' function. 
2. Distances between populations are calculated as the difference between the lengths of the population ID's (keys in alleles and pops dictionaries) since these track population history. As such, a larger difference = greater distance = lower migration rate. 
3. Distances and migration rates are stored in a dictionary ('Dis'), which is re-created at the start of every generation since more populations may have been generated. The key's in this dictionary are concatenated strings formed from the union of all pairwise combinations of existing populations. This makes it easy to determine the correct migration rate between all pairs of populations.


In [41]:
# Modules used throughout script
import random
from collections import OrderedDict
import csv
import time
from datetime import datetime
import os
import itertools
import math
import numpy as np

# Randomly sample 'N' alleles from lists containing alleles for locus A 
def sample_population_A(locus_A, N):
    new_locus_A = [random.choice(locus_A) for _ in range(N)]
    return new_locus_A

# Randomly sample 'N' alleles from lists containing alleles for locus B
def sample_population_B(locus_B, N):
    new_locus_B = [random.choice(locus_B) for _ in range(N)]
    return new_locus_B

# Function used to determine migration rates based on distances between all populations. 
def migration_rate(Distance_Dic):
    for Dkey, Dvalue in Distance_Dic.items():
        Mig_prop = 0.1 - 0.025*Dvalue[0] # Currently, this needs to be modified depending on the desired relationship between distance and migration
        Distance_Dic[Dkey].append(Mig_prop) # Append migration rate to 'Dis' dictionary
        
# Migration function for locus A. 
def migrate_A(Akey, Avalue, Dis, alleles, pop_list):
    if len(pop_list) > 1: # Migration only occurs if there is more than one population
        for i in pop_list:
            if Akey == i: # Do not migrate within populations
                pass
            else:
                con = Akey + '_' + i # Create concatenated string from current focal population and all other populations in 'pop_list', one at a time.
                if con in Dis.keys(): # Search for concatenated string in 'Dis' dictionary
                    mig_ind = int(math.ceil(Dis[con][1]*alleles[i]['S'][0])) # Calculate the number of alleles to migrate from migration rate and population size. Determined from source population
                    locus_A_keep = (sample_population_A(Avalue['A'], Avalue['S'][0] - mig_ind)) # Create list of alleles in focal population to be kept. Random sample of existing alleles. Complement of number of migratory individuals
                    locus_A_rep = [random.choice(alleles[i]['A']) for _ in range(mig_ind)] # Create list of replacement alleles by sampling alleles from source population. 
                    Avalue['A'] = locus_A_keep + locus_A_rep # Combine both kept and replaced alleles and 
        return Avalue['A']
    else:
        #If there is only one population, then resampling of existing alleles occurs (i.e. drift)
        Avalue['A'] = sample_population_A(Avalue['A'], Avalue['S'][0])
        return Avalue['A']
    
# Migration function for locus B. 
def migrate_B(Akey, Avalue, Dis, alleles, pop_list):
    if len(pop_list) > 1:
        for i in pop_list:
            if Akey == i:
                pass
            else:
                con = Akey + '_' + i
                if con in Dis.keys():
                    mig_ind = int(math.ceil(Dis[con][1]*alleles[i]['S'][0]))
                    locus_B_keep = (sample_population_A(Avalue['B'], Avalue['S'][0] - mig_ind))
                    locus_B_rep = [random.choice(alleles[i]['B']) for _ in range(mig_ind)]
                    Avalue['B'] = locus_B_keep + locus_B_rep
        return Avalue['B']
    else:
        Avalue['B'] = sample_population_B(Avalue['B'], Avalue['S'][0])
        return Avalue['B']


# From list containing alleles, calculate the frequency of 'A' or 'B' allele. 
def allele_freq(locus):
    p = sum(1*i.isupper() for i in locus)/float(len(locus))
    return p

# Function for logistic population growth. Takes current population size (from alleles dictionary) as input and return new population
# size. 
def pop_growth(r, Akey, Avalue, alleles):
    size = Avalue['S'][0] # Retrieve size of population. 'Akey' allows indexing of alleles dictionary in cline function
    R = r - (0.01*size) # Calculate modified growth rate. Modifier constant currently needs to be changed manually based on desired carrying capacity
    new_size = [int(math.ceil(R*size))] # Calculate new population size
    return new_size # Update population size in alleles dictionary

# Simple bottleneck function. Uses desired bottleneck proportion and returns an integer of number of alleles to sample based on population size.
# Only used when creating populations
def bottle(bot, Akey, Avalue):
    return int(math.ceil(bot*Avalue['S'][0]))

# Create new population as empty list and add to 'pops' dictionary. Also create two lists of alleles
# sampled from pool of alleles from population that generated the new one. Alleles added to 'alleles'
# dictionary
def create_population(p, Akey, Avalue, pops, alleles, bot): #'Akey' and 'Avalue' arguments local variables defined in 'cline' function
    if not alleles['1']['A']: #If there are no alleles for first population, pass. Only valid for first iteration when the populations have yet to be initialized
        #print 'There are no populations from which to sample!!'
        pass
    else:
        prob_list = (['1'] * int(10*p) ) + (['0'] * int(round(10*(1-p)))) #List with 0's and 1's based on argument 'p'
        create = [random.choice(prob_list) for _ in range(1)] #Randomly select an element from 'prob_list'
        if create[0] == '1': #If a '1' is sampled, create population
            global pop_counter #global variable that tracks the number of populations created
            pop_counter += 1 #Increment 'pop_counter' by 1 if population is being created. 
            #Add two lists to 'alleles' dictionary ('A' and 'B'). Naming: 'Pop.number_Population that created the new one'
            alleles['{0}.{1}'.format(pop_counter,Akey)] = {'A':sample_population_A(alleles[Akey].items()[0][1], bottle(bot, Akey, Avalue)),'B':sample_population_B(alleles[Akey].items()[2][1], bottle(bot, Akey, Avalue)), 'S':[bottle(bot, Akey, Avalue)]}
            pops['{0}.{1}'.format(pop_counter,Akey)] = [] #Empty list for new population. Naming same as alleles.
            
# Given the frequencies of 'A' and 'B' alleles, return the frequency of the 'acyanogenic' phenotype (i.e. recessive
# at either the A locus, B locus, or both) 
def phenotype(pA, pB):
    qA = 1-pA
    qB = 1-pB
    mut= qA**2 + qB**2 - (qA**2 * qB**2)
    WT = 1-mut
    return mut # Frequency of acyanogenic phenotype

# Cline function. Every generation, 'N' alleles are sampled for pool of alleles sampled the previous generation.
# Ever generation, every population has some probability (p) of generating a new population, with alleles
# sampled from the population that created it.
def cline(locus_A,locus_B, steps, N, p, pops, alleles, bot):
        for i in range(steps):
            Dis = {'{0}_{1}'.format(key1,key2):[abs(len(key2.split('.')) - len(key1.split('.')))] for key1,key2 in itertools.permutations(alleles.iterkeys(),2)}
            pop_list = alleles.keys()
            migration_rate(Dis)
            for Akey, Avalue in alleles.items():
                if Akey in pops.keys():
                    if 'A' and 'B' in Avalue.keys():
                        if not Avalue['A'] and not Avalue['B']: 
                            #If allele lists are empty, sample from list of initial allele frequencies. Only used for first generation
                            Avalue['S'] = [N]
                            Avalue['A'] = (sample_population_A(locus_A, N))
                            Avalue['B'] = (sample_population_B(locus_B, N))
                        else:
                            #If allele lists are not empty, sample from previously sampled set of alleles. 
                            Avalue['S'] = pop_growth(r, Akey, Avalue, alleles)
                            Avalue['A'] = (migrate_A(Akey, Avalue, Dis, alleles, pop_list))
                            Avalue['B'] = (migrate_B(Akey, Avalue, Dis, alleles, pop_list))
                    create_population(p, Akey, Avalue, pops, alleles, bot) #Create population. Alleles will be sampled (see above). Population is currently empty list
            for Pkey in pops.keys():
                #Calculate allele and phetype frequencies for every population, including newly created ones. 
                pA = allele_freq(alleles[Pkey].items()[0][1]) 
                pB = allele_freq(alleles[Pkey].items()[2][1])
                pops[Pkey].append([alleles[Pkey].items()[1][1][0], i, pA, pB, phenotype(pA, pB)])
        #return pops

        
# Using the functions defined above, 'simulate' performs 'sims' iterations of the cline function -- simulating the effects 
# of drift in a stepping stone model -- each time storing the results.
def simulate(pA, pB, steps, N, sims):
    qA = 1-pA # Frequency of 'a' allele
    qB = 1-pB
    # Make the two lists based on the allele frequency to represent the initial population
    locus_A = (['A'] * int(N*pA) ) + (['a'] * int(round(N*qA)) ) # [A,A,A,A,a,a,a,a,....]
    locus_B = (['B'] * int(N*pB) ) + (['b'] * int(round(N*qB)) ) 
    ####### sims simulations #####################
    # We will simulate 'steps' iterations of resampling this population to simulate drift
    # We will then repeat that simulation of 'steps' iterations 1000 times to get a mean
    ##############################################
    for s in range(sims):
        pops = OrderedDict({'1':[]}) # Re-initialize dictionary to store populations
        alleles = OrderedDict({'1':{'A':[],'B':[],'S':[]}}) # Re-initialize dictionary to store allele lists
        global pop_counter # Reset population counter
        pop_counter = 1
        # reset the population for each iteration. I don't actually think this is necessary
        locus_A = (['A'] * int(N*pA) ) + (['a'] * int(round(N*qA)) )  # Re-initialize initial allele lists.
        locus_B = (['B'] * int(N*pB) ) + (['b'] * int(round(N*qB)) ) 
        cline(locus_A,locus_B, steps, N, p, pops, alleles, bot) # Run cline function
        global sim
        sim[s] = pops # Append results to global 'sim' dictionary
        


In [36]:
N = 100 # Starting population size (i.e. sample 10 alleles)
pA = 0.5 # Initial frequency of 'A' allele
pB = 0.5 # Initial frequency of 'B' allele
qA = 1 - pA # 'a' allele
qB = 1 - pB # 'b' allele
steps = 10 # Number of generations
sims = 1 # Number of simulations
bot = 0.1 # Bottleneck proportion
pop_counter = 1 # Count of number of populations being created. Modified by 'create_population'. Used in naming populations
p = 0.1 # Probability of creating a population. 
r = 2 # Growth rate
sim = {} # Global dictionary used for storing results of each iteration
simulate(pA, pB, steps, N, sims) # Simulate function. 
sim

{0: OrderedDict([('1',
               [[100, 0, 0.46, 0.58, 0.41656176000000006],
                [100, 1, 0.39, 0.6, 0.472564],
                [100, 2, 0.45, 0.48, 0.49110400000000004],
                [100, 3, 0.4, 0.45, 0.5536000000000001],
                [100, 4, 0.37, 0.42, 0.59978284],
                [100, 5, 0.53, 0.4, 0.5013759999999999],
                [100, 6, 0.45, 0.31, 0.63457975],
                [100, 7, 0.5, 0.36, 0.5571999999999999],
                [100, 8, 0.55, 0.37, 0.5190277499999999],
                [100, 9, 0.61, 0.28, 0.59165136]]),
              ('2.1',
               [[10, 2, 0.5, 0.5, 0.4375],
                [19,
                 3,
                 0.631578947368421,
                 0.631578947368421,
                 0.2530444057366043],
                [35,
                 4,
                 0.5428571428571428,
                 0.4857142857142857,
                 0.4181964181591004],
                [58,
                 5,
                 0.58

In [93]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [28]:
pops = OrderedDict({'1':[]}) # Re-initialize dictionary to store populations
alleles = OrderedDict({'1':{'A':[],'B':[],'S':[]}}) # Re-initialize dictionary to store allele lists
N = 10 # Starting population size (i.e. sample 10 alleles)
pA = 0.5 # Initial frequency of 'A' allele
pB = 0.5 # Initial frequency of 'B' allele
qA = 1 - pA # 'a' allele
qB = 1 - pB # 'b' allele
steps = 5 # Number of generations
sims = 2 # Number of simulations
bot = int(round(N*0.1))
pop_counter = 1 # Count of number of populations being created. Modified by 'create_population'. Used in naming populations
p = 0.3 # Probability of creating a population. 
locus_A = (['A'] * int(N*pA) ) + (['a'] * int(round(N*qA)) ) # [A,A,A,A,a,a,a,a,....]
locus_B = (['B'] * int(N*pB) ) + (['b'] * int(round(N*qB)) )
r = 2.0

In [29]:
cline(locus_A,locus_B, steps, N, p, pops, alleles, bot)

In [31]:
for key, value in alleles.items():
    print key, value

1 [[10, 0, 0.7, 0.6, 0.23560000000000006], [19, 1, 0.7368421052631579, 0.631578947368421, 0.19558628310095844], [35, 2, 0.6857142857142857, 0.7142857142857143, 0.1723448563098709], [58, 3, 0.5862068965517241, 0.6896551724137931, 0.25104732065473273], [83, 4, 0.6626506024096386, 0.5301204819277109, 0.3094648249355471]]
2.1 [[10, 0, 0.7, 0.4, 0.4176], [19, 1, 0.5789473684210527, 0.5263157894736842, 0.3618833495752795], [35, 2, 0.7142857142857143, 0.6285714285714286, 0.20832986255726782], [58, 3, 0.6724137931034483, 0.8103448275862069, 0.13942186698073328], [83, 4, 0.7469879518072289, 0.6867469879518072, 0.15586090793224652]]
3.1 [[19, 1, 0.7368421052631579, 0.42105263157894735, 0.381220217769968], [35, 2, 0.7142857142857143, 0.45714285714285713, 0.35226988754685556], [58, 3, 0.6379310344827587, 0.39655172413793105, 0.44750592409523227], [83, 4, 0.5662650602409639, 0.40963855421686746, 0.47108571329356547]]
4.2.1 [[19, 1, 0.6842105263157895, 0.6842105263157895, 0.18950130830794729], [35, 

In [15]:
Dis = {'{0}_{1}'.format(key1,key2):[abs(len(key2.split('.')) - len(key1.split('.')))] for key1,key2 in itertools.permutations(alleles.iterkeys(),2)}
pop_list = alleles.keys()
#migration_rate(Dis)
#for Akey, Avalue in alleles.items():
 #   migrate(Akey, Avalue, Dis, alleles, pop_list)

In [19]:
if len(pop_list) == 1:
    print pop_list
else:
    print 'WTF'

WTF


In [96]:
for i,j in itertools.permutations(rows,2):
    print i,j, abs(j - i)

0 1 1
0 2 2
0 3 3
0 4 4
1 0 1
1 2 1
1 3 2
1 4 3
2 0 2
2 1 1
2 3 1
2 4 2
3 0 3
3 1 2
3 2 1
3 4 1
4 0 4
4 1 3
4 2 2
4 3 1


In [104]:
Matrix = np.zeros((10,10))
Matrix

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [105]:
i,j = random.randint(0, len(Matrix) - 1), random.randint(0, len(Matrix) - 1)
Matrix[i,j] = 1
i, j, Matrix

(7, 7, array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]))

In [284]:
for (i,j),value in np.ndenumerate(Matrix):
    if value == 0:
        Matrix[i,j+1] = 
Matrix

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [273]:
np.where(Matrix == 1)

(array([8]), array([3]))

In [43]:
def Distance(Matrix):
    rows = [i for i in range(len(Matrix))]
    cols = [i for i in range(len(Matrix))]
    matelem = [(i,j) for i in rows for j in cols]
    dis = [[i,j] for i in matelem for j in matelem]
    for i in dis:
        if i[0][0] == i[1][0]:
            dis1 = abs((i[1][1] - i[1][0]) - (i[0][1] - i[0][0]))
            i.append(dis1)
        elif i[0][1] == i[1][1]:
            dis2 = abs((i[1][1] - i[1][0]) - (i[0][1] - i[0][0]))
            i.append(dis2)    
        else:
            dis3 = (((i[1][1] - i[0][1])**2) + ((i[1][0] - i[0][0])**2))**(0.5)
            i.append(dis3)
    global Distance_Dic
    Distance_Dic = {'{0}.{1}'.format(key1,key2):[key3] for key1,key2,key3 in dis}
    migration_rate(Distance_Dic)
    #return Distance_Dic

In [44]:
Distance(Matrix)

In [45]:
Distance_Dic

{'(4, 7).(7, 5)': [3.605551275463989, 0.009861218113400266],
 '(8, 2).(7, 8)': [6.082762530298219, -0.0520690632574555],
 '(9, 0).(6, 4)': [5.0, -0.024999999999999994],
 '(8, 4).(8, 2)': [2, 0.05],
 '(8, 8).(0, 5)': [8.54400374531753, -0.11360009363293827],
 '(1, 3).(0, 5)': [2.23606797749979, 0.04409830056250526],
 '(6, 8).(4, 8)': [2, 0.05],
 '(3, 2).(6, 3)': [3.1622776601683795, 0.020943058495790506],
 '(0, 9).(0, 4)': [5, -0.024999999999999994],
 '(1, 2).(1, 4)': [2, 0.05],
 '(2, 1).(3, 9)': [8.06225774829855, -0.10155644370746372],
 '(3, 6).(5, 0)': [6.324555320336759, -0.05811388300841899],
 '(7, 4).(6, 8)': [4.123105625617661, -0.003077640640441512],
 '(5, 3).(5, 4)': [1, 0.07500000000000001],
 '(7, 9).(1, 9)': [6, -0.05000000000000002],
 '(3, 6).(0, 9)': [4.242640687119285, -0.006066017177982122],
 '(0, 6).(9, 4)': [9.219544457292887, -0.13048861143232218],
 '(8, 3).(0, 9)': [10.0, -0.15],
 '(8, 4).(9, 3)': [1.4142135623730951, 0.06464466094067262],
 '(8, 7).(0, 0)': [10.630145

In [39]:
migration_rate(Distance_Dic)

In [23]:
for key, value in Distance_Dic.items():
    print value[0]

TypeError: 'float' object has no attribute '__getitem__'

In [89]:
X = 9
Y = 9
neighbors(3,6)

TypeError: neighbors() takes exactly 1 argument (2 given)

In [102]:
def neighbors(Matrix):
    X, Y = (len(Matrix) - 1), (len(Matrix) - 1)
    for (x,y),value in np.ndenumerate(Matrix):
        N = [(x2, y2) for x2 in range(x-1, x+2)
            for y2 in range(y-1, y+2)
                if (-1 < x <= X and
                -1 < y <= Y and
                (x != x2 or y != y2) and
                (0 <= x2 <= X) and
                (0 <= y2 <= Y))]
        for z in N:
            i, j = z[0], z[1]
            if Matrix[i, j] == 0.0:
                Matrix[i, j] = 1.0
    return Matrix  

In [103]:
neighbors(Matrix)

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]])

In [ ]:
def assign_neighbors(N, Matrix):
    for x in N:
        i, j = x[0], x[1]
    

In [110]:
for (x,y),value in np.ndenumerate(Matrix):
    N = [(x2, y2) for x2 in range(x-1, x+2)
        for y2 in range(y-1, y+2)
        if (-1 < x <= X and
        -1 < y <= Y and
        (x != x2 or y != y2) and
        (0 <= x2 <= X) and
        (0 <= y2 <= Y))]
    for z in N:
        i, j = z[0], z[1]
        if Matrix[i, j] == 0.0:
            Matrix[i, j] = 1.0
Matrix

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]])

In [96]:
Matrix

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]])